# Disease spread simulation with not random node selection
In this notebook the algorithm for the simulation is modified to be performed with not random node selection. Then the results are saved in the form of the lists.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import statistics

In [2]:
data=open("C:/Users/wojci/Documents/Studia/praca/DANE/dane.txt","r")
data=data.readlines()
contacts=[]
nodes=[]
times=[]
#OBRÓBKA DANYCH
for i in range(0,len(data)):
    row=data[i]
    row=row.split()
    nodes.append(int(row[1]))
    nodes.append(int(row[2]))
    times.append(int(row[0])/60/60/24)
nodes=set(nodes)
nodes=list(nodes)  #tak otrzymuję wszystkie wierzchołki jakie są w sieci podczas całego badania

for i in range(0,len(data)):
    row=data[i]
    row=row.split()
    contacts.append((int(row[0])//60/60/24,int(row[1]),int(row[2]))) #i teraz kontakty mamy w postaci (czas, node1,node2)

In [3]:
G1000=nx.Graph(time=0,S=92)
inf_prob=0.006
incubation_period=1
infectious_period=7

for i in nodes:
    G1000.add_node(i,Id=i,state="S",time_of_inf=False)

In [4]:
def clear_net():
    for i in G1000.nodes:
        G1000.nodes[i]['state']="S"
        G1000.nodes[i]['time_of_inf']=False
    G1000.graph['time']=0
    G1000.graph['S']=92

In [5]:
def new_exposed(node,time):
    G1000.graph['S']-=1
    G1000.nodes[node]['state']='E'
    G1000.nodes[node]['time_of_inf']=time

In [6]:
def end_of_inc(i):
    
    if G1000.nodes[i]['time_of_inf']+incubation_period<=G1000.graph['time']:
        G1000.nodes[i]['state']="I"
            
def passing_to_recover(i):
    
    if G1000.nodes[i]['time_of_inf']+incubation_period+infectious_period<=G1000.graph['time']:
            G1000.nodes[i]['state']="R"
            G1000.nodes[i]['time_of_inf']=False
            
decision_dict={'E':end_of_inc,'I':passing_to_recover}
 
def transformations(): #te funkcje to modyfikacje żeby szybciej szło
    
    for i in G1000.nodes:
        try:
            func=decision_dict[G1000.nodes[i]['state']]
            func(i)
        except KeyError:
            pass

In [7]:
def IS(node1,node2,time):
    pr=random.random() 
    if pr<=inf_prob: #udało się zarazić
        new_exposed(node2,time)
        
def SI(node1,node2,time):
    pr=random.random() 
    if pr<=inf_prob: #udało się zarazić
        new_exposed(node1,time)
        
decision_dict2={'IS':IS,'SI':SI}
        
def new_contact(node1,node2,time):
    try:
        func=decision_dict2[G1000.nodes[node1]['state']+G1000.nodes[node2]['state']] 
        func(node1,node2,time)
    except KeyError:
            pass

In [8]:
def simple_epidemics(node): #used for MC simulation
    clear_net()
    new_exposed(node,0)
    for i in contacts:
        G1000.graph['time']=i[0]
        transformations()
        new_contact(i[1],i[2],i[0])
    return G1000.graph['S']

In [51]:
#Don't run it unless necessary! (long run time)
def monte_carlo(node):
    N=1000
    S=[92-simple_epidemics(node) for j in range(N)]
    chance_for_node=[i for i in S if i>=3]
    chance=len(chance_for_node)/N
    mean=statistics.mean(S)
    return [node,chance,mean]
    #return S

a=[monte_carlo(i) for i in G1000.nodes]


In [ ]:
%store a

In [ ]:
def monte_carlo_random(node):
    N=1000
    S=[92-simple_epidemics(random.random()) for j in range(N)]

In [52]:
S_153=monte_carlo(153) 


In [53]:
%store S_153

Stored 'S_153' (list)
